In [1]:
try:
    import polars as pl
except ImportError:
    !pip install polars
    import polars as pl

### Add Coordinates to Stations 

In [2]:
# Loading data
df_all = pl.read_parquet("./Data/Train/Train_2years.parquet")
df_all

Searched Station,Date,Train Nr.,Train Type,Departing Station,Scheduled Departure Time,Arrival Station,Scheduled Arrival Time,Actual Arrival Time,Delay,Cancellation,On Time,Departure Hour,Arrival Hour,Day of Week,Month,Year,Station Latitude,Station Longitude
str,date,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,i32,i32,i8,i8,i8,i8,i32,f64,f64
"""Landeck_Zams""",2024-09-30,"""RJX 869""","""RJX""","""Bregenz""",2024-09-30 12:40:00,"""Landeck-Zams""",2024-09-30 14:25:00,2024-09-30 14:27:00,2,0,1,12,14,1,9,2024,47.148287,10.578534
"""Wien_Hbf""",2025-01-08,"""RJ 130""","""RJ""","""Venezia Santa Lucia""",2025-01-08 15:52:00,"""Wien Hbf""",2025-01-08 23:36:00,2025-01-09 00:10:00,34,0,0,15,23,3,1,2025,48.185332,16.37865
"""Attnang_Puchheim""",2024-07-05,"""WB 917""","""WB""","""Salzburg Hbf""",2024-07-05 12:52:00,"""Attnang-Puchheim""",2024-07-05 13:35:00,2024-07-05 13:35:00,0,0,1,12,13,5,7,2024,48.012244,13.720598
"""Tullnerfeld""",2024-01-20,"""NJ 446""","""NJ""","""Wien Hbf (Autoreisezuganlage)""",2024-01-20 22:44:00,"""Tullnerfeld""",2024-01-20 23:19:00,2024-01-20 23:20:00,1,0,1,22,23,6,1,2024,48.295004,15.996219
"""Schärding""",2024-02-28,"""IC 296""","""IC""","""Fürth Hbf""",2024-02-28 00:33:00,"""Schärding""",2024-02-28 21:25:00,2024-02-28 21:25:00,0,0,1,0,21,3,2,2024,48.46422,13.442111
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wien_Meidling""",2023-06-26,"""NJ 490""","""NJ""","""Wien Hbf""",2023-06-26 19:35:00,"""Wien Meidling""",2023-06-26 19:40:00,2023-06-26 19:41:00,1,0,1,19,19,1,6,2023,48.174652,16.333905
"""Wels_Hbf""",2023-05-25,"""RJX 42""","""RJX""","""Mosonmagyarovar""",2023-05-25 19:20:00,"""Wels Hbf""",2023-05-25 22:00:00,2023-05-25 22:00:00,0,0,1,19,22,4,5,2023,48.165701,14.026241
"""St._Pölten_Hbf""",2024-12-18,"""EN 406""","""EN""","""Zebrzydowice""",2024-12-18 04:26:00,"""St.Pölten Hbf""",2024-12-18 22:41:00,2024-12-18 23:20:00,39,0,0,4,22,3,12,2024,48.208325,15.62464


In [3]:
# Load GTFS
stops_23 = pl.read_csv("./Data/Train/GTFS/2023/stops.txt")
stops_24 = pl.read_csv("./Data/Train/GTFS/2024/stops.txt")
trips_23  = pl.read_csv("./Data/Train/GTFS/2023/trips.txt")
trips_24  = pl.read_csv("./Data/Train/GTFS/2024/trips.txt")
stop_times_23 = pl.read_csv("./Data/Train/GTFS/2023/stop_times.txt")
stop_times_24 = pl.read_csv("./Data/Train/GTFS/2024/stop_times.txt")
shapes_23 = pl.read_csv("./Data/Train/GTFS/2023/shapes.txt")
shapes_24 = pl.read_csv("./Data/Train/GTFS/2024/shapes.txt")
calendar_23 = pl.read_csv("./Data/Train/GTFS/2023/calendar.txt")
calendar_24 = pl.read_csv("./Data/Train/GTFS/2024/calendar.txt")
calendar_25 = pl.read_csv("./Data/Train/GTFS/calendar.txt")
calendar_dates_23 = pl.read_csv("./Data/Train/GTFS/2023/calendar_dates.txt")
calendar_dates_24 = pl.read_csv("./Data/Train/GTFS/2024/calendar_dates.txt")
calendar_dates_25 = pl.read_csv("./Data/Train/GTFS/calendar_dates.txt")

In [4]:
# Getting Station Coordinates
station_coords= pl.concat([stops_23.select(["stop_name", "stop_lat", "stop_lon"]), stops_24.select(["stop_name", "stop_lat", "stop_lon"])])

station_coords = station_coords.group_by("stop_name").agg([
    pl.first("stop_lat").alias("Station Latitude"),
    pl.first("stop_lon").alias("Station Longitude")
])

replace_map = {
    "Hauptbahnhof": "Hbf",
    " Bahnhof": "",
    " Bahnhst": "",
    " (Bahnsteige 1-2)": "",
    " 1": "",
}

def replace_text(text):
    for old, new in replace_map.items():
        text = text.replace(old, new)
    return text
    
station_coords = station_coords.with_columns(
    pl.col("stop_name")
    .map_elements(replace_text, return_dtype = pl.String)
)

### Join Data sets

In [5]:
# Prepare Station for joining
replace_map = {
    "Wörther See": "Wörthersee",
    "Linz": "Linz/Donau",
    "Amstetten NÖ": "Amstetten",
    "Pörtschach am Wörthersee": "Pörtschach",
    "Spittal-Millstättersee": "Spittal-Millstätter See",
    "Neumarkt am Wallersee":  "Neumarkt",
    "Ötztal": "Ötztal-Bahnhof",
    "Langen am Arlberg": "Langen a. A.",
    "Stadt Rottenmann": "Rottenmann",
    "Feldbach/Raab": "Feldbach",
    "St.Michael in Obersteiermark": "St.Michael",
    "St. Michael": "St.Michael",
    "St.Veit/Glan": "St. Veit an der Glan",	
    "St. Anton am Arlberg": "St. Anton a. A.",
    "St.Pölten Hbf": "St. Pölten Hbf",
    "St. Johann im Pongau": "St.Johann",
    "Linz/DonauHbf": "Linz/Donau Hbf",
    "Krumpendorf/Wörthersee": "Krumpendorf",
    "Seekirchen am Wallersee": "Seekirchen",
    "Graz Ostbahnhof-Messe": "Graz Hbf",
    "Kirchberg in Tirol": "Kirchberg i. T.",
    "Golling/Salzach-Abtenau": "Golling-Abtenau",
    "Micheldorf": "Micheldorf in OÖ",
    "Salzburg Süd": "Salzburg Hbf",
    "Altenmarkt im Pongau": "Altenmarkt",
    "Friesach in Kärnten": "Friesach",
    "Graz Hbf (Busbahnhof)": "Graz Hbf",
    "Lienz in Osttirol": "Lienz",
    "Stein/Enns": "Stein an der Enns",
    "Schwarzach im Pongau-St.Veit": "Schwarzach-St.Veit",
    "Hallwang-Elixhausen": "Hallwang-Elixhausen S-Bahn",
    "St. Johann in Tirol": "St. Johann i. T.",
    "Lasberg-St. Oswald": "Lasberg-St.Oswald",
    "Hopfgarten im Brixental": "Hopfgarten i. Br.",
    "Seefeld in Tirol": "Seefeld i. T.",
    "Wien Hbf (Autoreisezuganlage)": "Wien Hbf",
    "Wien Hbf (Autoreisezug)": "Wien Hbf",
    "Vöcklabruck Bahnhof": "Vöcklabruck",
    "Klaus an der Pyhrnbahn": "Klaus/Pyhrnbahn",
    "Altmünster am Traunsee": "Altmünster/Traunsee",
    "Wien Matzleinsdorfer Platz (ÖBB)": "Wien Hbf",
    "Haus im Ennstal": "Haus",
    "Baden b.Wien": "Baden",
    "Micheldorf in OÖ in OÖ": "Micheldorf in OÖ",
    "Neunkirchen NÖ": "Neunkirchen",
    "Bad Mitterndorf Heilbrunn": "Bad Mitterndorf",
    "Graz Don Bosco": "Graz Hbf",
    "Kuchl": "Kuchl S-Bahn",
    "Eben im Pongau": "Eben",
    "Graz-Liebenau Murpark": "Graz Hbf",
    "Ebensee/Traunsee": "Ebensee",
    "Linz/Donau(Donau)Hbf": "Linz/Donau Hbf",
    "Rosenbach bei Villach": "Rosenbach",
    "Graz Hbf (Bahnsteige 1-2)": "Graz Hbf",
    "Wien Hbf (Bahnsteige 1-2)": "Wien Hbf",
    "St.Egyden am Steinfeld": "St. Egyden/Stfd",	
    "Rohr im Kremstal-Bad Hall": "Rohr-Bad Hall",
    "Puch b.Hallein Urstein": "Puch Urstein (FH)",
    "Steeg/Hallstätter See-Gosau": "Steeg-Gosau",
    "Lend/Salzach": "Lend",
    "Wien Mitte": "Wien Hbf",
    "Kirchberg am Wagram": "Kirchberg am Wagram Bhf.",
    "Deutsch Wagram": "Deutsch-Wagram",
    "Wien Hbf-Landstraße": "Wien Hbf",
    "Limberg NÖ-Maissau": "Limberg-Maissau",
    "Lambach Markt": "Lambach OÖ",
    "Bruck/Glocknerstraße-Fusch": "Bruck-Fusch",
    "Graz Liebenau Murpark (Bahnhst)": "Graz Hbf",
    "Wolkersdorf im Weinviertel": "Wolkersdorf",
    "Studenzen-Fladnitz im Raabtal": "Studenzen-Fladnitz",
    "Mixnitz Bärenschützklamm": "Mixnitz-Bärenschützklamm",
    "Salzburg Taxham Europark": "Salzburg Hbf",
    "Salzburg Parsch": "Salzburg Hbf",
    "Tauplitz Schiflugschanze Kulm": "Tauplitz",
    "Pernegg/Mur": "Pernegg",
    "Bad Vigaun": "Bad Vigaun S-Bahn",
    "Puch bei Hallein": "Puch Urstein (FH)",
    "Unter Purkersdorf": "Purkersdorf Zentrum",
    "Salzburg Mülln-Altstadt": "Salzburg Hbf",
    "Salzburg Aiglhof": "Salzburg Hbf",
    "Krems an der Donau": "Krems",
    "Hallein Burgfried": "Hallein",
    "St.Johann-Weistrach": "St. Johann-Weistrach",
    "Kuchl S-Bahn Garnei": "Kuchl S-Bahn",
    "Gmunden Engelhof": "Gmunden",
    "Rohrendorf b.Krems": "Rohrendorf/Krems",
    "Purkersdorf b.Wien Zentrum": "Purkersdorf Zentrum",
    "Purkersdorf b.Wien Sanatorium": "Purkersdorf Zentrum",
    "Salzburg Sam": "Salzburg Hbf",
    "Leonding": "Leonding Westbahn",
    "Pasching": "Pasching b.Linz",
    "Eugendorf": "Eugendorf S-Bahn",
    "Kirchstetten b.Neulengbach": "Kirchstetten",
    "Wallersee": "Wallersee S-Bahn",
    "Lambach": "Lambach OÖ",
    "Salzburg Liefering": "Salzburg Hbf",
    "Salzburg Kasern": "Salzburg Hbf",
    "St.Peter-Seitenstetten": "St. Peter-Seitenstetten Bhf.",
    "Köstendorf Weng": "Köstendorf Weng S-Bahn",
    "Hetzmannsdorf-Wullersdorf": "Hetzmannsdorf-Wullersdorf Bhf.",
    "Neuhaus/Gail": "Neuhaus an der Gail",
    "Himberg b.Wien": "Himberg",
    "Lambach OÖ OÖ": "Lambach OÖ",
    "Linz/Donau-Pichling": "Linz/Donau Pichling",
    "Salzburg Gnigl": "Salzburg Hbf",
    "Elsbethen": "Elsbethen S-Bahn",
    "Götzendorf/Leitha": "Götzendorf"
}

def replace_text(text):
    for old, new in replace_map.items():
        text = text.replace(old, new)
    return text

df_all = df_all.with_columns(
    pl.col("Arrival Station")
    .map_elements(replace_text, return_dtype = pl.String)
    .alias("stop_name")
)

In [6]:
df_all = df_all.join(
    station_coords.unique("stop_name"),
    on = "stop_name",
    how = "left"
).drop(pl.col("stop_name"))

# Include Destination

In [7]:
# Get service_id and corresponding dates
calendar_all = pl.concat(
    [calendar_23, calendar_24, calendar_25]
).with_columns(
    pl.col("start_date").cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d"),
    pl.col("end_date").cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d")
)

# Expand schadule on every date
calendar_all = calendar_all.with_columns(
    pl.date_ranges("start_date", "end_date").alias("date")
).explode("date").drop(
    ["start_date", "end_date"]
).with_columns(
    pl.col("date").dt.weekday().alias("weekday")
)

calendar_all = calendar_all.with_columns(
    pl.concat_arr([
        pl.col("monday"), pl.col("tuesday"), pl.col("wednesday"),
        pl.col("thursday"), pl.col("friday"), pl.col("saturday"),
        pl.col("sunday")
    ]).alias("days")
)

# Keep only the dates, that are in the schedule
calendar_all = calendar_all.with_columns(
    pl.col("days").arr.get(pl.col("weekday") - 1).alias("get")
).filter(pl.col("get") == 1).drop(["monday", "tuesday",	"wednesday", "thursday", "friday", "saturday", "sunday", "weekday", "days", "get"])

In [8]:
# In- and Exclude special days 
calendar_dates_all = pl.concat([calendar_dates_23, calendar_dates_24, calendar_dates_25]).with_columns(
    pl.col("date").cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d")
)

added = calendar_dates_all.filter(pl.col("exception_type") == 1).select(["service_id", "date"])
removed = calendar_dates_all.filter(pl.col("exception_type") == 2).select(["service_id", "date"])

final_calendar = calendar_all.join(removed, on=["service_id", "date"], how = "anti")
final_calendar = final_calendar.vstack(added).unique()
final_calendar

service_id,date
str,date
"""TA+00x20""",2025-12-02
"""TA+add40""",2023-11-06
"""TA+0sb20""",2024-10-12
"""TA+ma430""",2024-12-08
"""TA+vs900""",2024-03-31
…,…
"""TA+juw20""",2024-12-08
"""TA+5c630""",2024-09-16
"""TA+f3730""",2024-11-04


### Join Data sets

In [10]:
# Combingin trip with shape with dates
trips = pl.concat([trips_23, trips_24])
trips_with_date = trips.select(
    ["service_id", "trip_headsign",  "trip_short_name"]
).join(final_calendar, on = "service_id", how = "left")

In [12]:
df_gtfs = df_all.join(
    trips_with_date,
    left_on = ["Train Nr.", "Date"],
    right_on = ["trip_short_name", "date"],
    how = "left"
).unique()

In [13]:
df_gtfs.head(2)

Searched Station,Date,Train Nr.,Train Type,Departing Station,Scheduled Departure Time,Arrival Station,Scheduled Arrival Time,Actual Arrival Time,Delay,Cancellation,On Time,Departure Hour,Arrival Hour,Day of Week,Month,Year,Station Latitude,Station Longitude,service_id,trip_headsign
str,date,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,i32,i32,i8,i8,i8,i8,i32,f64,f64,str,str
"""Mallnitz_Obervellach""",2024-09-12,"""EN 480""","""EN""","""Rijeka""",2024-09-11 18:57:00,"""Mallnitz-Obervellach""",2024-09-12 01:00:00,null,null,1,0,18,1,4,9,2024,46.98109,13.176282,"""TA+9s030""","""Stuttgart Hauptbahnhof"""
"""Wien_Hbf""",2024-05-11,"""IC 1031""","""IC""","""Praha hl.n.""",2024-05-11 06:15:00,"""Wien Hbf""",2024-05-11 10:21:00,2024-05-11 10:25:00,4,0,1,6,10,6,5,2024,48.185332,16.37865,null,null


In [14]:
df_all.write_parquet("./Data/Train/Train_2years.parquet")
df_gtfs.write_parquet("./Data/Train/Train_2years_headsign.parquet")

# Work in Progress: Shape and distance travelled

In [9]:
# Getting Shape information -> too much data crashes server
#shapes = pl.concat([shapes_23, shapes_24])
#shapes = (
#    shapes.sort("shape_pt_sequence")
#    .group_by("shape_id")
#    .agg([
#        pl.struct(
#            [
#                pl.col("shape_pt_lat").alias("lat"),
#                pl.col("shape_pt_lon").alias("lon")
#            ]
#        ).alias("Shape Coordinates"),
#        pl.concat_list(pl.col("shape_dist_traveled")).alias("Distance covered")
#    ])
#)

In [11]:
# Combining trips with shape
#trips_with_all = trips_with_calendar.join(shapes, on = "shape_id", how = "left")
#trips_with_all